# Automação Web e Busca de Informações com Python


Trabalhamos em uma importadora e o preço dos nossos produtos é vinculado a cotação de:
- Dólar
- Euro
- Ouro

Pegar na internet, de forma automática, a cotação desses 3 itens e saber quanto devemos cobrar pelos nossos produtos, considerando uma margem de contribuição que temos na nossa base de dados.

Base de Dados: https://drive.google.com/drive/folders/1KmAdo593nD8J9QBaZxPOG1yxHZua4Rtv?usp=sharing

In [6]:
#Entrar na internet
from selenium import webdriver
from selenium.webdriver.common.keys import Keys

navegador = webdriver.Chrome()

#Pegar cotação do dólar

navegador.get("https://www.google.com.br/")
navegador.find_element_by_xpath('/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input').send_keys("Cotação dólar")
navegador.find_element_by_xpath('/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input').send_keys(Keys.ENTER)
cotacao_dolar = navegador.find_element_by_xpath('//*[@id="knowledge-currency__updatable-data-column"]/div[1]/div[2]/span[1]').get_attribute("data-value")
print(cotacao_dolar)

#Pegar cotação do euro

navegador.get("https://www.google.com.br/")
navegador.find_element_by_xpath('/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input').send_keys("Cotação euro")
navegador.find_element_by_xpath('/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input').send_keys(Keys.ENTER)
cotacao_euro = navegador.find_element_by_xpath('//*[@id="knowledge-currency__updatable-data-column"]/div[1]/div[2]/span[1]').get_attribute("data-value")
print(cotacao_euro)

#Pegar cotação do ouro

navegador.get("https://www.melhorcambio.com/ouro-hoje")
cotacao_ouro = navegador.find_element_by_xpath('/html/body/div[6]/div[1]/div/div/input[2]').get_attribute("value")
cotacao_ouro = cotacao_ouro.replace(",",".")
print(cotacao_ouro)

5.2282
6.141697743000001
294.77


In [7]:
#Importar a base de dados
import pandas as pd

tabela = pd.read_excel("Produtos.xlsx")
display(tabela)

,Produtos,Preço Base Original,Moeda,Cotação,Preço Base Reais,Margem,Preço Final
0,Câmera Canon,999.99,Dólar,5,4999.95,1.40,6999.930
1,Carro Renault,4500.00,Euro,6,27000.00,2.00,54000.000
2,Notebook Dell,899.99,Dólar,5,4499.95,1.70,7649.915
3,IPhone,799.00,Dólar,5,3995.00,1.70,6791.500
4,Carro Fiat,3000.00,Euro,6,18000.00,1.90,34200.000
5,Celular Xiaomi,480.48,Dólar,5,2402.40,2.00,4804.800
6,Joia 20g,20.00,Ouro,350,7000.00,1.15,8050.000


In [8]:
#Atualizando os valores da coluna "Cotação"

tabela.loc[tabela["Moeda"] == "Dólar", "Cotação"] = float(cotacao_dolar) #Em relação ao "Dólar"
tabela.loc[tabela["Moeda"] == "Euro", "Cotação"] = float(cotacao_euro) #Em relação ao "Euro"
tabela.loc[tabela["Moeda"] == "Ouro", "Cotação"] = float(cotacao_ouro) #Em relação ao "Ouro"

#Atualizando os valores da coluna "Preço Base Reais"

tabela["Preço Base Reais"] = tabela["Preço Base Original"] * tabela["Cotação"]

#Atualizando os valores da coluna "Preço Final"

tabela["Preço Final"] = tabela["Preço Base Reais"] * tabela["Margem"]
tabela["Preço Final"] = tabela["Preço Final"].map("{:.2f}".format)

display(tabela)

,Produtos,Preço Base Original,Moeda,Cotação,Preço Base Reais,Margem,Preço Final
0,Câmera Canon,999.99,Dólar,5.228200,5228.147718,1.40,7319.41
1,Carro Renault,4500.00,Euro,6.141698,27637.639844,2.00,55275.28
2,Notebook Dell,899.99,Dólar,5.228200,4705.327718,1.70,7999.06
3,IPhone,799.00,Dólar,5.228200,4177.331800,1.70,7101.46
4,Carro Fiat,3000.00,Euro,6.141698,18425.093229,1.90,35007.68
5,Celular Xiaomi,480.48,Dólar,5.228200,2512.045536,2.00,5024.09
6,Joia 20g,20.00,Ouro,294.770000,5895.400000,1.15,6779.71


In [ ]:
# Exportar a base de dados atualizada

tabela.to_excel("Produtos Novo.xlsx")

navegador.quit()